# Synchronization for MEA Recording

This notebook takes a MCD file that include the analog signal to recovery the synchronization time of all stimuli. It's very important step to analyze with precision the response of the recording cells.

This codes create files with synchronization time for each frame, repeated frame time and a csv file with all relevant information about the stimuli.

CSV file has a list of events, a event is a sequence of together images. Each event has a field of start and end time, number of frames, total time of the event, repeated frames and time inter event.

In [ ]:
from configparser import ConfigParser, ExtendedInterpolation

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from spikelib.preprocessing import Sync
from spikelib.utils import check_directory

%matplotlib notebook 
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

In [ ]:
experiment_name = config['EXP']['name']
mcd_file = config['FILES']['raw']
output_folder = config['SYNC']['folder']
output_folder

## Get synchronization signal
To analyze synchronization signal we use a specific class in **mealib.preprocessing** called Sync. With this we can real MCD file and processing all information or use analyzed info previously.

input:   
* **mcd_file**: name of MCD file
* **mcd_channel**: number of analog channel in MCD file
* **real_fps**: the real number of frame per second on screen (look at log file)

output: 3 attributes of the class   
* **start_end_frame**: synchronization time for each frame
* **repeated_frame**: repeated frame time
* **total_duration**: total duration of MCD record

Note:
Analog signal only register the start of frame, so the end time in a sequence of images is extrapolate to have the correct time in the output file.

[example](https://tinyurl.com/ydxlvds9)

For more information look at [here](https://github.com/creyesp/SamplingInterface)


In [ ]:
real_fps = 59.7233

sync_data = Sync(experiment_name, real_fps)
sync_data.read_mcd(mcd_file)
#Show electrodes in MCD file
sync_data.show_entities()

Choose analog channel from index entites

In [ ]:
mcd_channel = 0
sync_data.analyzer(mcd_channel)

# Event list
## Create event list
From start_end_frame attribute created above we can compute the difference between all frames and get only frames with a difference more than one frame to create a event.


<img src="https://raw.githubusercontent.com/creyesp/SamplingInterface/master/Doc/img/img%20examples/analog_signal_sequence_img.png" alt="Events" width="600px"/>

In [ ]:
sync_data.create_events()
sync_data.event_list.head()

## Add repeated frame in the event lists
Some time the screen repeat one frame because the time to prepare the next frame wasn't reached and the screen show again the last frame. It should be critic in some case because the synchronization is lost in this case. If you know exactly the repeated frame you can correct the stimulus or discard this event.

<img src="https://raw.githubusercontent.com/creyesp/SamplingInterface/master/Doc/img/img%20examples/repeated_frame.png" alt="drawing" width="400px"/>

In [ ]:
sync_data.add_repeated()
sync_data.event_list.head()

**Note:** The times in event_duration have small differences with the theoretic times, it's because the theatrical FPS of the projector is different to the real FPS.   

For example, projector show theoretically 60 FPS but the real FPS is 59.7523. Please check it in the log file. 
* 2100 images to 60 [fps] = 35 [s]
* 2100 images to 59.7523 [fps] = 35.145 [s]
The differences could be significant!

## Check repeated
There is a method to show analog signal in a specific window of time, we can use it to check repeated frame using the above information.

In [ ]:
start_point = 48210517-4000
window = 4000
sync_data.plot_window(mcd_channel, start_point, window)

## Save sync to file

In [ ]:
check_directory(output_folder)

sync_data.save_analyzed(output_folder)
sync_data.save_events(output_folder)

# Create individual sync files
For each event we can create a separated sync file using event information in csv. Before you have check the csv file and remove all false positive and you should write the name of events to do more easy understand them.

Create a definitive version removing the last underscore name. '*_.csv' -> '*.csv'


In [ ]:
source_folder = config['SYNC']['folder']
sync_file = config['SYNC']['events']
output_folder = source_folder + '/event_list/'
check_directory(output_folder)

sync_data.load_analyzed(source_folder)
sync_data.load_events(sync_file)
sync_data.create_separated_sync(output_folder)


# Close file

In [ ]:
sync_data.close_file()